In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      Cython>=0.22 and NumPy are required.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pystan
ERROR: Could not build wheels for pystan, which is required to install pyproject.toml-based projects

[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      /private/var/folders/_5/6v34wcl528z8xh_w0f3x9dxw0000gq/T/pip-install-_bvnrgz0/fbprophet_959daecef3b74a75835a2270e6a0c8c7/setup.py:10: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        from pkg_resources import (

In [2]:
 !pip install prophet


[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [4]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-02-15,"$2,777.60","$2,865.85","$2,764.01","$2,824.38","$2,824.38",23734481937
2024-02-16,"$2,825.48","$2,858.45","$2,760.33","$2,803.69","$2,803.69",17057114638
2024-02-17,"$2,803.74","$2,805.13","$2,724.39","$2,786.67","$2,786.67",17932379943
2024-02-18,"$2,786.71","$2,892.84","$2,767.91","$2,879.00","$2,879.00",23355830478
2024-02-19,"$2,881.30","$2,983.37","$2,860.26","$2,943.57","$2,943.57",15163110589


In [5]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2294 entries, 2017-11-09 to 2024-02-19
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2294 non-null   float64
 1   High       2294 non-null   float64
 2   Low        2294 non-null   float64
 3   Close      2294 non-null   float64
 4   Adj Close  2294 non-null   float64
 5   Volume     2294 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 125.5 KB


In [6]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [8]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [9]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds",
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [10]:
df.tail()

,ds,y
2289,2024-02-15,"$2,777.60"
2290,2024-02-16,"$2,825.48"
2291,2024-02-17,"$2,803.74"
2292,2024-02-18,"$2,786.71"
2293,2024-02-19,"$2,881.30"


In [11]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [12]:
m = Prophet(
    seasonality_mode="multiplicative"
)

m.fit(df)

13:03:38 - cmdstanpy - INFO - Chain [1] start processing
13:03:38 - cmdstanpy - INFO - Chain [1] done processing


In [13]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2654,2025-02-14
2655,2025-02-15
2656,2025-02-16
2657,2025-02-17
2658,2025-02-18


In [14]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2654,2025-02-14,"$3,199.83","$1,780.76","$4,709.47"
2655,2025-02-15,"$3,194.76","$1,740.56","$4,640.38"
2656,2025-02-16,"$3,208.08","$1,726.55","$4,777.16"
2657,2025-02-17,"$3,214.66","$1,714.09","$4,798.53"
2658,2025-02-18,"$3,205.37","$1,736.01","$4,731.88"


In [15]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

2392.2547173125213

In [16]:
plot_plotly(m, forecast)

In [17]:
plot_components_plotly(m, forecast)